In [1]:
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import re
from geopy.geocoders import Nominatim, Photon
import math

### PLAN

1. Pobranie danych
2. Przejrzeć co mamy
    - wielkosc data setu
    - jakie kolumny
    - jakiego typu dane
    - podejrzec probke
    - jesli dane sa we wlasciwym formacie: proste statystyki
3. Konwersja danych na odpowiedni format
4. Co z brakujacymi wartosciami?

#### Zaimportuj i przejrzyj dane

In [2]:
dane = pd.read_csv('data/DataWorkshop.csv')

In [3]:
print('Rozmiar danych: ', dane.shape)

Rozmiar danych:  (229929, 30)


In [4]:
print('Kolumny w datasecie: ')
dane.columns

Kolumny w datasecie: 


Index(['Data zdarzenia', 'Godzina', 'GPS x', 'GPS y', 'Miejscowość', 'Ulica',
       'Numer domu', 'Numer drogi', 'KM HM (Pikietaż)', 'Obszar',
       'Skrzyżowanie', 'Geometria drogi', 'Charakterystyka miejsca zdarzenia',
       'Oświetlenie', 'Warunki atmosferyczne', 'Rodzaj zdarzenia', 'Wina',
       'Rodzaj drogi', 'Nawierzchnia drogi', 'Stan nawierzchni',
       'Sygnalizacja świetlna', 'Oznakowanie poziome',
       'Liczba pasów w zdarzeniu', 'Zachowanie kierującego',
       'Zachowanie pieszego', 'Inne przyczyny', 'Liczba rannych w zdarzeniu',
       'Liczba zabitych w zdarzeniu', 'Liczba kolizji', 'Liczba wypadków'],
      dtype='object')

In [5]:
print('Typy danych: ')
dane.dtypes

Typy danych: 


Data zdarzenia                        object
Godzina                               object
GPS x                                 object
GPS y                                 object
Miejscowość                           object
Ulica                                 object
Numer domu                            object
Numer drogi                           object
KM HM (Pikietaż)                      object
Obszar                                object
Skrzyżowanie                          object
Geometria drogi                       object
Charakterystyka miejsca zdarzenia     object
Oświetlenie                           object
Warunki atmosferyczne                 object
Rodzaj zdarzenia                      object
Wina                                  object
Rodzaj drogi                          object
Nawierzchnia drogi                    object
Stan nawierzchni                      object
Sygnalizacja świetlna                 object
Oznakowanie poziome                   object
Liczba pas

In [6]:
print('Brakujace wartosci: ')

missing = dane.isnull().sum()
print(missing)

Brakujace wartosci: 
Data zdarzenia                            0
Godzina                                   0
GPS x                                     0
GPS y                                     0
Miejscowość                               0
Ulica                                 20728
Numer domu                            93884
Numer drogi                               0
KM HM (Pikietaż)                     194888
Obszar                                    1
Skrzyżowanie                         166779
Geometria drogi                       66008
Charakterystyka miejsca zdarzenia         2
Oświetlenie                               0
Warunki atmosferyczne                     1
Rodzaj zdarzenia                          0
Wina                                      0
Rodzaj drogi                              1
Nawierzchnia drogi                        1
Stan nawierzchni                          1
Sygnalizacja świetlna                     0
Oznakowanie poziome                       2
Liczba pasó

In [7]:
print('Brakujace wartosci jako procent zbioru, %: ')
index=[]
lst=[]
for item in missing.iteritems():
    index.append(item[0])
    lst.append(round((item[1]/dane.shape[0])*100,4))
missing_percentage=pd.Series(lst, index=index)
print(missing_percentage)

Brakujace wartosci jako procent zbioru, %: 
Data zdarzenia                        0.0000
Godzina                               0.0000
GPS x                                 0.0000
GPS y                                 0.0000
Miejscowość                           0.0000
Ulica                                 9.0150
Numer domu                           40.8317
Numer drogi                           0.0000
KM HM (Pikietaż)                     84.7601
Obszar                                0.0004
Skrzyżowanie                         72.5350
Geometria drogi                      28.7080
Charakterystyka miejsca zdarzenia     0.0009
Oświetlenie                           0.0000
Warunki atmosferyczne                 0.0004
Rodzaj zdarzenia                      0.0000
Wina                                  0.0000
Rodzaj drogi                          0.0004
Nawierzchnia drogi                    0.0004
Stan nawierzchni                      0.0004
Sygnalizacja świetlna                 0.0000
Oznakowanie

In [8]:
# Probka danych
dane.sample(50)

,Data zdarzenia,Godzina,GPS x,GPS y,Miejscowość,Ulica,Numer domu,Numer drogi,KM HM (Pikietaż),Obszar,...,Sygnalizacja świetlna,Oznakowanie poziome,Liczba pasów w zdarzeniu,Zachowanie kierującego,Zachowanie pieszego,Inne przyczyny,Liczba rannych w zdarzeniu,Liczba zabitych w zdarzeniu,Liczba kolizji,Liczba wypadków
43514,2016-11-02,20:35:00,18*53'413,50*09'526,MIKOŁÓW,KONSTYTUCJI 3 MAJA,67,P5302S,NaN,Obszar zabudowany,...,Brak,Jest,2.0,Nieprawidłowe wyprzedzanie,NaN,NaN,0,0,1,0
38192,2016-10-03,11:20:00,19*09'461,50*34'375,RZENISZÓW,NaN,NaN,1,"503,5",Obszar niezabudowany,...,Brak,Jest,NaN,Nieustąpienie pierwszeństwa przejazdu,NaN,NaN,0,0,1,0
35243,2016-09-15,15:10:00,18*38'380,50*19'081,GLIWICE,PORTOWA,19,P7210S,NaN,Obszar zabudowany,...,Brak,Jest,2.0,Niezachowanie bezp. odleg. między pojazdami,NaN,NaN,0,0,1,0
207498,2019-08-29,07:39:00,18*57'264,50*17'216,CHORZÓW,ASTRÓW,18,G210008S,NaN,Obszar zabudowany,...,Brak,Nie ma,2.0,Nieustąpienie pierwszeństwa przejazdu,NaN,NaN,0,0,1,0
9425,2016-03-13,16:40:00,19*07'471,49*50'192,KOZY,BIELSKA,93,52,"12,3",Obszar zabudowany,...,Brak,Jest,NaN,Nieprawidłowe: wyprzedzanie,NaN,NaN,0,0,1,0
133141,2018-05-28,12:30:00,19*08'079,50*14'249,MYSŁOWICE,WOJSKA POLSKIEGO,8,G240047S,NaN,Obszar zabudowany,...,Brak,Nie ma,2.0,Inne przyczyny,NaN,NaN,0,0,1,0
34832,2016-09-08,15:00:00,18*41'109,49*45'503,GUMNA,NaN,NaN,S52B,629,Obszar niezabudowany,...,Brak,Jest,6.0,Inne przyczyny,NaN,NaN,0,0,1,0
7793,2016-03-02,08:55:00,18*39'556,50*17'411,GLIWICE,PLEBAŃSKA,4,I2,NaN,Obszar zabudowany,...,Brak,Nie ma,1.0,Nieudzielenie pierwszeństwa przejazdu,NaN,NaN,0,0,1,0
120543,2018-03-07,06:50:00,19*06'310,50*17'189,SOSNOWIEC,GEN.STEFANA GROTAROWECKIEGO,130,P7631S,NaN,Obszar niezabudowany,...,Brak,Jest,1.0,Niezachowanie bezp. odleg. między pojazdami,NaN,NaN,0,0,1,0
136615,2018-06-20,18:00:00,19*07'514,50*48'421,CZĘSTOCHOWA,MIROWSKA,NaN,P6616S,NaN,Obszar zabudowany,...,"Jest, działa",Jest,2.0,Nieustąpienie pierwszeństwa przejazdu,NaN,NaN,0,0,1,0


### Konwersja typu danych

In [9]:
# Utworzenie kopii zbioru danych. Dalsza konwersja odbywac sie bedzie na kopii danych
df=dane.copy()
df.rename(columns={'Data zdarzenia': 'Data', 'GPS x': 'GPSx', 'GPS y': 'GPSy', 'Miejscowość':'Miejscowosc',
                   'Numer domu':'NrDomu','Numer drogi':'NrDrogi','KM HM (Pikietaż)':'Pikietaz', 'Skrzyżowanie':'Skrzyzowanie',
                  'Geometria drogi': 'GeometriaDrogi', 'Charakterystyka miejsca zdarzenia': 'CharMiejsca',
                  'Oświetlenie':'Oswietlenie', 'Warunki atmosferyczne':'Pogoda', 'Rodzaj zdarzenia':'RodzajZdarzenia',
                  'Rodzaj drogi':'RodzajDrogi', 'Nawierzchnia drogi':'NawierzchniaDrogi', 'Stan nawierzchni': 'StanNawierzchni',
                  'Sygnalizacja świetlna':'SygnalizacjaSwietlna', 'Oznakowanie poziome':'ZnakiPoziome',
                  'Liczba pasów w zdarzeniu':'LiczbaPasow','Zachowanie kierującego':'ZachowanieKierowcy',
                  'Zachowanie pieszego':'ZachowaniePieszego', 'Inne przyczyny':'InnePrzyczyny', 'Liczba rannych w zdarzeniu':'Ranni',
                  'Liczba zabitych w zdarzeniu':'Zabici', 'Liczba kolizji':'Kolizja', 'Liczba wypadków':'Wypadek'}, inplace=True)


#### Data i czas

In [10]:
# Data zdarzenia i Godzina scalone do jednej kolumny
print('kolumna DATA ZDARZENIA: ', df['Data'].dtypes)
print('kolumna GODZINA: ', df['Data'].dtypes)
df['data_full'] = df['Data'] + ' ' + df['Godzina']
df.sample(10)

kolumna DATA ZDARZENIA:  object
kolumna GODZINA:  object


,Data,Godzina,GPSx,GPSy,Miejscowosc,Ulica,NrDomu,NrDrogi,Pikietaz,Obszar,...,ZnakiPoziome,LiczbaPasow,ZachowanieKierowcy,ZachowaniePieszego,InnePrzyczyny,Ranni,Zabici,Kolizja,Wypadek,data_full
96653,2017-10-11,15:40:00,19*07'081,50*18'120,BĘDZIN,NaN,NaN,86,"17,2",Obszar zabudowany,...,Jest,NaN,Niezachowanie bezp. odleg. między pojazdami,NaN,NaN,0,0,1,0,2017-10-11 15:40:00
182625,2019-03-28,08:40:00,19*14'288,50*28'485,SIEWIERZ,KIELECKA,NaN,78,NaN,Obszar zabudowany,...,Jest,2.0,Nieustąpienie pierwszeństwa przejazdu,NaN,NaN,2,0,0,1,2019-03-28 08:40:00
154994,2018-10-14,14:25:00,18*53'020,49*37'056,WISŁA,GŁĘBCE,32D,W941,NaN,Obszar zabudowany,...,Nie ma,2.0,Nieprawidłowe wyprzedzanie,NaN,NaN,1,0,0,1,2018-10-14 14:25:00
194635,2019-06-06,16:10:00,18*24'552,50*03'252,RYDUŁTOWY,ROMUALDA TRAUGUTTA,NaN,P5024S,NaN,Obszar zabudowany,...,Jest,2.0,Nieprawidłowe wymijanie,NaN,NaN,0,0,1,0,2019-06-06 16:10:00
45186,2016-11-15,12:50:00,18*35'315,50*06'578,RYBNIK,MIKOŁOWSKA,NaN,W925,NaN,Obszar niezabudowany,...,Jest,NaN,Niezachowanie bezp. odleg. między pojazdami,NaN,NaN,0,0,1,0,2016-11-15 12:50:00
138969,2018-07-06,10:25:00,18*32'438,50*06'001,RYBNIK,GLIWICKA,NaN,P7014S,NaN,Obszar zabudowany,...,Jest,2.0,Nieustąpienie pierwszeństwa przejazdu,NaN,NaN,0,0,1,0,2018-07-06 10:25:00
57655,2017-02-01,14:00:00,18*42'160,50*17'279,GLIWICE,PRZEWOZOWA,32,GMINNA,NaN,Obszar zabudowany,...,Nie ma,1.0,Nieprawidłowe cofanie,NaN,NaN,0,0,1,0,2017-02-01 14:00:00
26878,2016-07-12,12:45:00,18*41'322,49*57'439,PAWŁOWICE,KRUCZA,10,P4163S,NaN,Obszar zabudowany,...,Nie ma,NaN,Nieprawidłowe cofanie,NaN,NaN,0,0,1,0,2016-07-12 12:45:00
11554,2016-03-31,17:05:00,18*32'422,50*06'429,RYBNIK,GLIWICKA,45,I2,NaN,Obszar zabudowany,...,Jest,NaN,Nieudzielenie pierwszeństwa przejazdu,NaN,NaN,0,0,1,0,2016-03-31 17:05:00
198037,2019-06-27,22:00:00,18*36'093,49*53'531,ZEBRZYDOWICE,JULIUSZA SŁOWACKIEGO,58B,W937,"8,7",Obszar zabudowany,...,Jest,2.0,Nieprawidłowe zmienianie pasa ruchu,NaN,NaN,0,0,1,0,2019-06-27 22:00:00


In [11]:
# Data zdarzenia i Godzina do typu datetime
df['data_full'] = pd.to_datetime(df['data_full'], format='%Y-%m-%d %H:%M:%S')
df.sample(10)
# TO-DO: Jaki to dzien tygodnia? Jaki to dzien miesiaca? To dzien czy noc? Jaka faza ksiezyca itd..

,Data,Godzina,GPSx,GPSy,Miejscowosc,Ulica,NrDomu,NrDrogi,Pikietaz,Obszar,...,ZnakiPoziome,LiczbaPasow,ZachowanieKierowcy,ZachowaniePieszego,InnePrzyczyny,Ranni,Zabici,Kolizja,Wypadek,data_full
169865,2019-01-05,13:25:00,19*08'529,50*22'138,SARNÓW,GŁÓWNA,23,P4719S,NaN,Obszar zabudowany,...,Nie ma,2.0,Nieprawidłowe cofanie,NaN,NaN,0,0,1,0,2019-01-05 13:25:00
29003,2016-08-02,05:45:00,18*51'240,50*08'595,ŁAZISKA GÓRNE,PSTROWSKIEGO,19,P5326S,NaN,Obszar zabudowany,...,Jest,2.0,Nieustąpienie pierwszeństwa przejazdu,NaN,NaN,0,0,1,0,2016-08-02 05:45:00
116549,2018-02-04,19:45:00,19*11'290,50*15'422,SOSNOWIEC,NaN,NaN,S1,"542,3",Obszar niezabudowany,...,Jest,2.0,Niedostosowanie prędkości do warunków ruchu,NaN,NaN,0,0,1,0,2018-02-04 19:45:00
148160,2018-09-05,07:05:00,18*28'525,50*00'144,WODZISŁAW ŚLĄSKI,GRODZISKO,1,P5018S,NaN,Obszar niezabudowany,...,Jest,2.0,Nieprawidłowe wymijanie,NaN,NaN,0,0,1,0,2018-09-05 07:05:00
161789,2018-11-23,18:50:00,18*41'320,50*02'491,ŻORY,MĘCZENNIKÓW OŚWIĘCIMSKICH,22,P9019S,NaN,Obszar zabudowany,...,Nie ma,2.0,Gwałtowne hamowanie,NaN,NaN,0,0,1,0,2018-11-23 18:50:00
64521,2017-03-22,08:10:00,18*42'458,50*16'026,GLIWICE,PSZCZYŃSKA,NaN,44,NaN,Obszar zabudowany,...,Jest,2.0,Nieprawidłowe zmienianie pasa ruchu,NaN,NaN,0,0,1,0,2017-03-22 08:10:00
31713,2016-08-20,17:00:00,19*07'278,50*13'322,MYSŁOWICE,OBRZEŻNA ZACHODNIA,NaN,P6490S,NaN,Obszar niezabudowany,...,Jest,NaN,Niezachowanie bezp. odleg. między pojazdami,NaN,NaN,0,0,1,0,2016-08-20 17:00:00
221980,2019-11-16,14:55:00,18*38'499,50*13'568,KNURÓW,SZPITALNA,37,G370002S,NaN,Obszar zabudowany,...,Jest,3.0,Nieprawidłowe zmienianie pasa ruchu,NaN,NaN,0,0,1,0,2019-11-16 14:55:00
6100,2016-02-13,15:50:00,18*55'521,50*21'041,BYTOM,SZAFRANKA,NaN,G150218S,NaN,Obszar zabudowany,...,Jest,NaN,Nieprzestrzeganie innych sygnałów,NaN,NaN,0,0,1,0,2016-02-13 15:50:00
198207,2019-06-29,09:10:00,18*59'306,49*54'067,CZECHOWICE-DZIEDZICE,NaN,NaN,1,"593,9",Obszar niezabudowany,...,Jest,4.0,Niezachowanie bezp. odleg. między pojazdami,NaN,NaN,0,0,1,0,2019-06-29 09:10:00


#### Wspolrzedne geograficzne

Domyslny zapis wspolrzednych to D\*M'S([D]egree, [M]inute, [S]cond) np. 18\*53'289. Jednak, gdy pierwsza cyfra sekund jest zero, czasem wystepuje zapis D\*M''S np. 18\*53''11. Wystepowaly takze bledne zapisy takie jak 4\*\*56'569 i 50\*09'\*01.

In [12]:
def dmsToDecimal(coordinate_as_string):
    splittedCoordinate=re.findall(r"\w+", coordinate_as_string)
    degrees = float(splittedCoordinate[0])
    minutes = float(splittedCoordinate[1])
    if len(splittedCoordinate)==3:
        seconds = float(splittedCoordinate[2])/10
    else:
        seconds = 0
    coordinate_as_float = degrees + minutes/60 + seconds/(60*60)
    return coordinate_as_float

In [13]:
# Przygotowana wyzej funkcje wywoluje na calych kolumnach dancych
df['GPSx'] = df['GPSx'].apply(lambda x: dmsToDecimal(x))
df['GPSy'] = df['GPSy'].apply(lambda x: dmsToDecimal(x))
df.sample(10)
# TO-DO Wyszukac i wykorzystac biblioteke okreslajaca wspolrzedne GPS na podstawie adresu
# TO-DO Popraw punkty ktore wypadly poza woj. slaskie bazujac na ulicy i numerze

,Data,Godzina,GPSx,GPSy,Miejscowosc,Ulica,NrDomu,NrDrogi,Pikietaz,Obszar,...,ZnakiPoziome,LiczbaPasow,ZachowanieKierowcy,ZachowaniePieszego,InnePrzyczyny,Ranni,Zabici,Kolizja,Wypadek,data_full
113183,2018-01-17,08:20:00,18.962694,50.154583,TYCHY,PODLESKA,51,P8006S,NaN,Obszar zabudowany,...,Nie ma,2.0,Niedostosowanie prędkości do warunków ruchu,NaN,NaN,0,0,1,0,2018-01-17 08:20:00
51358,2016-12-21,10:10:00,18.640972,49.746028,CIESZYN,BOLESŁAWA CHROBREGO,NaN,P2701S,NaN,Obszar zabudowany,...,Jest,NaN,Nieustąpienie pierwszeństwa przejazdu,NaN,NaN,0,0,1,0,2016-12-21 10:10:00
130281,2018-05-11,06:45:00,18.916917,50.162972,MIKOŁÓW,BIELSKA,NaN,W928,NaN,Obszar niezabudowany,...,Jest,4.0,Nieprawidłowe cofanie,NaN,NaN,0,0,1,0,2018-05-11 06:45:00
224671,2019-12-03,04:00:00,19.362583,50.923444,KŁOMNICE,CZĘSTOCHOWSKA,110A,91,NaN,Obszar zabudowany,...,Jest,2.0,Nieustąpienie pierwszeństwa przejazdu,NaN,NaN,0,0,1,0,2019-12-03 04:00:00
199024,2019-07-03,11:30:00,19.134861,50.281000,SOSNOWIEC,3 MAJA,NaN,P7619S,NaN,Obszar zabudowany,...,Jest,2.0,Nieustąpienie pierwszeństwa przejazdu,NaN,NaN,0,0,1,0,2019-07-03 11:30:00
130469,2018-05-11,16:35:00,18.953389,50.382500,PIEKARY ŚLĄSKIE,GEN.JERZEGO ZIĘTKA,60,G280082S,NaN,Obszar zabudowany,...,Jest,2.0,Niezachowanie bezp. odleg. między pojazdami,NaN,NaN,0,0,1,0,2018-05-11 16:35:00
134633,2018-06-07,09:50:00,19.005528,49.912917,CZECHOWICE-DZIEDZICE,JÓZEFA PIŁSUDSKIEGO,8,GMINNA,NaN,Obszar zabudowany,...,Nie ma,2.0,Nieprawidłowe cofanie,NaN,NaN,0,0,1,0,2018-06-07 09:50:00
20804,2016-06-02,15:05:00,19.162889,50.768917,CZĘSTOCHOWA,ALEJA WOJSKA POLSKIEGO,NaN,1,"480,4",Obszar zabudowany,...,Jest,NaN,Niezachowanie bezp. odl. między pojazdami,NaN,NaN,0,0,1,0,2016-06-02 15:05:00
220601,2019-11-08,11:57:00,18.936722,50.264583,CHORZÓW,STEFANA BATOREGO,NaN,P9405S,NaN,Obszar zabudowany,...,Jest,2.0,Niezachowanie bezp. odleg. między pojazdami,NaN,NaN,0,0,1,0,2019-11-08 11:57:00
163023,2018-12-01,18:25:00,18.909222,50.491694,MIASTECZKO ŚLĄSKIE,METALOWA,8,G657004S,NaN,Obszar zabudowany,...,Nie ma,2.0,Niedostosowanie prędkości do warunków ruchu,NaN,NaN,0,0,1,0,2018-12-01 18:25:00


In [14]:
# Czy wszystkie punkty znajduja sie w granicach wojewodztwa slaskiego?
# Współrzędne geograficzne skrajnych punktów:
# północny: 51°05′57″ szer. geogr. N
# południowy: 49°23′38″ szer. geogr. N
# zachodni: 18°02′06″ dług. geogr. E
# wschodni: 19°58′26″ dług. geogr. E
northBorder = dmsToDecimal("51*05'57")
southBorder = dmsToDecimal("49*23'38")
westBorder = dmsToDecimal("18*02'06")
eastBorder = dmsToDecimal("19*58'26")

df[(df['GPSx']>eastBorder) | (df['GPSx']<westBorder) | (df['GPSy']>northBorder) | (df['GPSy']<southBorder)]

,Data,Godzina,GPSx,GPSy,Miejscowosc,Ulica,NrDomu,NrDrogi,Pikietaz,Obszar,...,ZnakiPoziome,LiczbaPasow,ZachowanieKierowcy,ZachowaniePieszego,InnePrzyczyny,Ranni,Zabici,Kolizja,Wypadek,data_full
15349,2016-04-27,13:45:00,19.984250,50.269083,KATOWICE,TRASA NIKODEMA I JÓZEFA RENCÓW,NaN,W902,NaN,Obszar niezabudowany,...,Jest,NaN,Niedostosowanie prędkości do warunków ruchu,NaN,NaN,0,0,1,0,2016-04-27 13:45:00
20566,2016-06-02,10:55:00,17.640861,50.335556,GLIWICE,LITERATÓW,69,I2,NaN,Obszar zabudowany,...,Nie ma,2.0,Inne przyczyny (od 11.2015),NaN,NaN,0,0,1,0,2016-06-02 10:55:00
27105,2016-07-17,18:30:00,19.389639,51.554361,OSINY,CZĘSTOCHOWSKA,52,791,NaN,Obszar zabudowany,...,Jest,NaN,Inne przyczyny,NaN,NaN,0,0,1,0,2016-07-17 18:30:00
32758,2016-08-16,12:50:00,15.676500,50.771389,KONIECPOL,RYNEK,NaN,794,NaN,Obszar zabudowany,...,Nie ma,NaN,Nieprawidłowe cofanie,NaN,NaN,0,0,1,0,2016-08-16 12:50:00
32880,2016-08-30,16:00:00,21.999083,50.341028,ZABRZE,NaN,NaN,A4,"318,1",Obszar niezabudowany,...,Jest,NaN,Niedostosowanie prędkości do warunków ruchu,NaN,NaN,0,0,1,0,2016-08-30 16:00:00
34895,2016-09-09,14:00:00,16.775611,50.618917,BONOWICE,NaN,86A,78,NaN,Obszar niezabudowany,...,Jest,NaN,Nieprawidłowe wyprzedzanie,NaN,NaN,0,0,1,0,2016-09-09 14:00:00
37291,2016-09-28,19:45:00,15.998528,50.252694,KATOWICE,NaN,NaN,A4,"335,9",Obszar niezabudowany,...,Jest,NaN,NaN,NaN,Niesprawność techniczna pojazdu,0,0,1,0,2016-09-28 19:45:00
45271,2016-11-16,07:20:00,19.995889,50.234028,KATOWICE,BRYNOWSKA,53,P6453S,NaN,Obszar zabudowany,...,Jest,NaN,Niezachowanie bezp. odleg. między pojazdami,NaN,NaN,0,0,1,0,2016-11-16 07:20:00
49595,2016-12-10,23:12:00,19.345528,51.205611,SKRZYDLÓW,GŁÓWNA,NaN,GMINNA,NaN,Obszar niezabudowany,...,Nie ma,NaN,NaN,NaN,Nieustalone,0,0,1,0,2016-12-10 23:12:00
50695,2016-12-16,19:05:00,19.315111,51.086361,ŁĘG,NaN,NaN,1,"447,8",Obszar niezabudowany,...,Jest,NaN,Nieprawidłowe zmienianie pasa ruchu,NaN,NaN,0,0,1,0,2016-12-16 19:05:00


#### Miejsce zdarzenia



In [15]:
# Ulice zapisywane sa w sposob niespojny np. Mickiewicza, A.Mickiewicza, Adama Mickiewicza
# Czy sa zapisywane spojnie wzgledem miejscowosci?
#df[df['Ulica'].str.contains('MICKIEWICZA',na=False)&df['Miejscowosc'].str.contains('KATOWICE',na=False)]
df[df['Ulica'].str.contains('MICKIEWICZA',na=False)&df['Miejscowosc'].str.contains('CZĘSTOCHOWA',na=False)]
# Wniosek: Zapis nazwy ulicy nie jest spojny nawet w ramach jednej miejscowosci

,Data,Godzina,GPSx,GPSy,Miejscowosc,Ulica,NrDomu,NrDrogi,Pikietaz,Obszar,...,ZnakiPoziome,LiczbaPasow,ZachowanieKierowcy,ZachowaniePieszego,InnePrzyczyny,Ranni,Zabici,Kolizja,Wypadek,data_full
3080,2016-01-22,12:40:00,19.117972,50.801083,CZĘSTOCHOWA,RONDO A.MICKIEWICZA,NaN,P6614S,NaN,Obszar zabudowany,...,Jest,NaN,Nieudzielenie pierwszeństwa przejazdu,NaN,NaN,0,0,1,0,2016-01-22 12:40:00
4814,2016-02-07,10:05:00,19.113667,50.800972,CZĘSTOCHOWA,MICKIEWICZA,25/31,G110525S,NaN,Obszar zabudowany,...,Nie ma,NaN,Nieprawidłowe: skręcanie,NaN,NaN,0,0,1,0,2016-02-07 10:05:00
4990,2016-02-08,18:30:00,19.115667,50.801167,CZĘSTOCHOWA,MICKIEWICZA,NaN,G110525S,NaN,Obszar zabudowany,...,Jest,NaN,Nieustąpienie pierwszeństwa pieszemu przy skrę...,NaN,NaN,1,0,0,1,2016-02-08 18:30:00
5013,2016-02-02,16:30:00,19.070306,50.480306,CZĘSTOCHOWA,MICKIEWICZA,NaN,G110525S,NaN,Obszar zabudowany,...,Jest,NaN,Nieudzielenie pierwszeństwa przejazdu,NaN,NaN,0,0,1,0,2016-02-02 16:30:00
5669,2016-02-12,17:30:00,19.118250,50.801333,CZĘSTOCHOWA,RONDO MICKIEWICZA,NaN,P6614S,NaN,Obszar zabudowany,...,Jest,NaN,Nieprawidłowe: zmienianie pasa ruchu,NaN,NaN,0,0,1,0,2016-02-12 17:30:00
8394,2016-03-03,16:15:00,19.117861,50.800694,CZĘSTOCHOWA,RONDO MICKIEWICZA,NaN,P6614S,NaN,Obszar zabudowany,...,Jest,NaN,Niezachowanie bezp. odl. między pojazdami,NaN,NaN,0,0,1,0,2016-03-03 16:15:00
14907,2016-04-21,14:55:00,19.113583,50.801000,CZĘSTOCHOWA,MICKIEWICZA,25/31,G110525S,NaN,Obszar zabudowany,...,Nie ma,NaN,Nieprawidłowe: cofanie,NaN,NaN,0,0,1,0,2016-04-21 14:55:00
18767,2016-05-17,12:35:00,19.117306,50.800667,CZĘSTOCHOWA,MICKIEWICZA,1,G110525S,NaN,Obszar zabudowany,...,Jest,NaN,Niedostosowanie prędkości do warunków ruchu,NaN,NaN,0,0,1,0,2016-05-17 12:35:00
19048,2016-05-20,09:35:00,19.117639,50.800778,CZĘSTOCHOWA,RONDO MICKIEWICZA,NaN,P6614S,NaN,Obszar zabudowany,...,Jest,NaN,Niezachowanie bezp. odl. między pojazdami,NaN,NaN,0,0,1,0,2016-05-20 09:35:00
20173,2016-05-27,12:10:00,19.117639,50.800806,CZĘSTOCHOWA,RONDO MICKIEWICZA,NaN,P6614S,NaN,Obszar zabudowany,...,Jest,NaN,Nieprawidłowe: zmienianie pasa ruchu,NaN,NaN,0,0,1,0,2016-05-27 12:10:00


In [16]:
# Pikietaz okresla miejsce zdarzenia na drodze poza terenem zabudowanym
#Zapisywany jako xx,yyy gdzie:
#xx – całkowita liczba kilometrów od początku drogi;
#yyy – całkowita liczba metrów liczona od ostatniego pełnego kilometra.
# W zbiorze danych prawdopodobnie jako wartosc w km, tyle ze z uzyciem "," jako separatora dziesietnego
def pikietazToFloat(pikietazRaw):
    if isinstance(pikietazRaw, str):
        pikietazAsFloat = float(pikietazRaw.replace(",", "."))
    else:
        pikietazAsFloat = pikietazRaw
    return pikietazAsFloat
df['Pikietaz']=df['Pikietaz'].apply(lambda x: pikietazToFloat(x))
df['Pikietaz']=df['Pikietaz'].fillna('')
df[df['Pikietaz'].notnull()].sample(10)

,Data,Godzina,GPSx,GPSy,Miejscowosc,Ulica,NrDomu,NrDrogi,Pikietaz,Obszar,...,ZnakiPoziome,LiczbaPasow,ZachowanieKierowcy,ZachowaniePieszego,InnePrzyczyny,Ranni,Zabici,Kolizja,Wypadek,data_full
132519,2018-05-19,17:30:00,19.141472,50.300861,SOSNOWIEC,NaN,45,94,,Obszar zabudowany,...,Jest,2.0,Nieustąpienie pierwszeństwa przejazdu,NaN,NaN,0,0,1,0,2018-05-19 17:30:00
73108,2017-05-18,17:55:00,18.649917,50.251722,GLIWICE,RYBNICKA,NaN,78,,Obszar niezabudowany,...,Jest,2.0,Niezachowanie bezp. odleg. między pojazdami,NaN,NaN,0,0,1,0,2017-05-18 17:55:00
39597,2016-10-11,01:40:00,19.049556,49.800806,BIELSKO-BIAŁA,PARTYZANTÓW,112,W942,,Obszar zabudowany,...,Jest,NaN,NaN,NaN,"Obiekty, zwierzęta na drodze",0,0,1,0,2016-10-11 01:40:00
57873,2017-02-02,09:58:00,18.955694,50.295889,CHORZÓW,KATOWICKA,NaN,79,,Obszar zabudowany,...,Jest,3.0,Niezachowanie bezp. odleg. między pojazdami,NaN,NaN,0,0,1,0,2017-02-02 09:58:00
175387,2019-02-08,23:50:00,19.096139,50.790111,CZĘSTOCHOWA,KOŚCIELNA,86,G110391S,,Obszar zabudowany,...,Nie ma,2.0,Nieustąpienie pierwszeństwa przejazdu,NaN,NaN,0,0,1,0,2019-02-08 23:50:00
88993,2017-09-01,08:15:00,18.854806,50.457583,TARNOWSKIE GÓRY,GRODZKA,NaN,P3291S,,Obszar zabudowany,...,Jest,2.0,Nieprzestrzeganie znaków i innych sygnałów,NaN,NaN,0,0,1,0,2017-09-01 08:15:00
126922,2018-04-18,17:11:00,19.217111,50.100028,CHEŁM ŚLĄSKI,GÓRNOŚLĄSKA,40,780,,Obszar zabudowany,...,Jest,2.0,NaN,NaN,"Obiekty, zwierzęta na drodze",0,0,1,0,2018-04-18 17:11:00
79687,2017-06-29,08:00:00,19.020222,50.254389,KATOWICE,JAGIELLOŃSKA,6,P6441S,,Obszar zabudowany,...,Jest,NaN,"Nieprawidłowe zatrzymywanie, postój",NaN,NaN,1,0,0,1,2017-06-29 08:00:00
117331,2018-02-13,13:25:00,18.724028,49.961944,WARSZOWICE,CIESZYŃSKA,4,81,,Obszar niezabudowany,...,Jest,2.0,Niezachowanie bezp. odleg. między pojazdami,NaN,NaN,0,0,1,0,2018-02-13 13:25:00
217272,2019-10-22,09:00:00,19.237528,50.318889,DĄBROWA GÓRNICZA,ŚW. JANA PAWŁA II,NaN,S1,535,Obszar niezabudowany,...,Jest,4.0,Nieprawidłowe zmienianie pasa ruchu,NaN,NaN,0,0,1,0,2019-10-22 09:00:00


In [17]:
# Zamiana wartosci NaN na puste stringi. Ulatwi to korzystanie z bibliotek do geolokacji
df['Pikietaz']=df['Pikietaz'].fillna('')
df['Miejscowosc'] = df['Miejscowosc'].fillna('')
df['Ulica'] = df['Ulica'].fillna('')
df['NrDomu'] = df['NrDomu'].fillna('')
df['NrDrogi'] = df['NrDrogi'].fillna('')

In [18]:
# Obszar: zabudowany/niezabudowany
# Przypisane zostaly wartosci logiczne: Obszar zabudowany-True, Obszar niezabudowany False.
# W razie potrzeby takie wartosci mozna wygodnie przeksztalcic na int o wartosciach odpowiednio 1 i 0
df['Obszar'].value_counts()
df['Obszar'] = df['Obszar'].apply(lambda x: True if x=='Obszar zabudowany' else False)
df.sample(10)

,Data,Godzina,GPSx,GPSy,Miejscowosc,Ulica,NrDomu,NrDrogi,Pikietaz,Obszar,...,ZnakiPoziome,LiczbaPasow,ZachowanieKierowcy,ZachowaniePieszego,InnePrzyczyny,Ranni,Zabici,Kolizja,Wypadek,data_full
63024,2017-03-07,19:50:00,18.321639,49.922417,CHAŁUPKI,BOGUMIŃSKA,26,G612903S,,True,...,Jest,NaN,Nieprawidłowe wyprzedzanie,NaN,NaN,0,0,1,0,2017-03-07 19:50:00
2880,2016-01-24,09:35:00,18.672556,49.910861,PIELGRZYMOWICE,GOLASOWICKA,,P4162S,,True,...,Jest,NaN,Niedostosowanie prędkości do warunków ruchu,NaN,NaN,0,0,1,0,2016-01-24 09:35:00
180862,2019-03-16,08:00:00,19.106250,50.817667,CZĘSTOCHOWA,ALEJA JANA PAWŁA II,106,I2,,True,...,Nie ma,2.0,Nieprawidłowe cofanie,NaN,NaN,0,0,1,0,2019-03-16 08:00:00
46826,2016-11-25,16:40:00,18.489833,50.046722,RADLIN,MARIACKA,,G530006S,,True,...,Jest,2.0,Nieustąpienie pierwszeństwa przejazdu,NaN,NaN,0,0,1,0,2016-11-25 16:40:00
96553,2017-10-10,19:30:00,18.551917,50.063306,RYBNIK,1 MAJA,55,W929,,True,...,Jest,NaN,Inne przyczyny,Inne przyczyny,NaN,1,0,0,1,2017-10-10 19:30:00
26316,2016-07-12,12:15:00,18.594139,49.949083,JASTRZĘBIE-ZDRÓJ,WIELKOPOLSKA,2B,I2,,True,...,Nie ma,NaN,Nieprawidłowe omijanie,NaN,NaN,0,0,1,0,2016-07-12 12:15:00
99215,2017-10-26,12:36:00,18.953528,49.989528,PSZCZYNA,,,1,,True,...,Jest,NaN,Nieprawidłowe zmienianie pasa ruchu,NaN,NaN,0,0,1,0,2017-10-26 12:36:00
226860,2019-12-14,12:45:00,18.529194,50.085028,RYBNIK,WODZISŁAWSKA,90,78,,True,...,Jest,2.0,Niezachowanie bezp. odleg. między pojazdami,NaN,NaN,0,0,1,0,2019-12-14 12:45:00
80814,2017-07-06,15:04:00,19.161361,50.598028,KOZIEGŁOWY,MONIUSZKI,1,G595070S,,True,...,Nie ma,NaN,Nieprawidłowe skręcanie,NaN,NaN,0,0,1,0,2017-07-06 15:04:00
129509,2018-05-06,16:30:00,18.973417,50.282306,KATOWICE,BOLESŁAWA CHROBREGO,37,P6418S,,True,...,Jest,2.0,"Nieprawidłowe zatrzymywanie, postój",NaN,NaN,0,0,1,0,2018-05-06 16:30:00


In [19]:
# Skrzyzowanie
# Zmiana nazwy na niezawierajaca polskich znakow
df['Skrzyzowanie'].unique()
df['Skrzyzowanie'].value_counts()
# Zamiana wartosci tekstowych na int zgodnie z zasada:
# 0 : O ruchu okrężnym; 1 : Równorzędne; 2 : Z drogą z pierwsz.
skrzyzowaniemap = {'O ruchu okrężnym':0, 'Równorzędne':1, 'Z drogą z pierwsz.':2}
df=df.applymap(lambda x: skrzyzowaniemap.get(x) if x in skrzyzowaniemap else x)

In [20]:
df['GeometriaDrogi'].value_counts()
# Geometria pozioma: 
# 0: Odcinek prosty; 1: Zakret(luk)

# Geometria pionowa: 
# 0: Spadek; 1: wzniesienie; 2: wierzcholek wzniesienia
# Skelic skrzyzowanie z geometria

Odcinek prosty                      142703
Zakręt, łuk                          16917
Spadek                                1866
Wzniesienie                           1317
Odcinek prosty; Spadek                 430
Odcinek prosty; Wzniesienie            329
Zakręt, łuk; Spadek                    179
Zakręt, łuk; Wzniesienie               108
Wierzchołek wzn.                        50
Odcinek prosty; Wierzchołek wzn.        16
Zakręt, łuk; Wierzchołek wzn.            5
Odcinek prosty; Zakręt, łuk              1
Name: GeometriaDrogi, dtype: int64

In [21]:
df['Geometria_drogi_poziom'] = df['GeometriaDrogi'].copy()
df['Geometria_drogi_poziom'] = df['Geometria_drogi_poziom'].apply(lambda x: 0 if 'Odcinek prosty' in str(x) else x)
df['Geometria_drogi_poziom'] = df['Geometria_drogi_poziom'].apply(lambda x: 1 if 'Zakręt' in str(x) else x)
df['Geometria_drogi_poziom'] = df['Geometria_drogi_poziom'].apply(lambda x: 1 if 'łuk' in str(x) else x)
# Co z reszta? Tzn z sytuacja kiedy nie jest zdefiniowane czy droga byla prosta czy byl to zakret?
df['Geometria_drogi_poziom'].value_counts()

0                   143479
1                    17209
Spadek                1866
Wzniesienie           1317
Wierzchołek wzn.        50
Name: Geometria_drogi_poziom, dtype: int64

In [22]:
df['Geometria_drogi_pion'] = df['GeometriaDrogi'].copy()
df['Geometria_drogi_pion'] = df['Geometria_drogi_pion'].apply(lambda x: 0 if 'Spadek' in str(x) else x)
df['Geometria_drogi_pion'] = df['Geometria_drogi_pion'].apply(lambda x: 1 if 'Wzniesienie' in str(x) else x)
df['Geometria_drogi_pion'] = df['Geometria_drogi_pion'].apply(lambda x: 2 if 'Wierzchołek wzn.' in str(x) else x)
# Co z reszta? Tzn z sytuacja kiedy nie jest zdefiniowane uksztaltowanie terenu?
df['Geometria_drogi_pion'].value_counts()

Odcinek prosty                 142703
Zakręt, łuk                     16917
0                                2475
1                                1754
2                                  71
Odcinek prosty; Zakręt, łuk         1
Name: Geometria_drogi_pion, dtype: int64

In [23]:
# Charakterystyka miejsca zdarzenia
df['CharMiejsca'].value_counts()

Jezdnia                                             191185
Parking, plac, MOP                                   17988
Pobocze                                               4676
Przejście dla pieszych                                3624
Chodnik, droga dla pieszych                           2612
Most, wiadukt, łącznica, tunel                        1796
Skarpa, rów                                           1356
Wjazd, wyjazd z posesji, pola                         1322
Roboty drogowe, oznakowanie tymczasowe                1235
Parking, plac, MOP (od 11.2015)                       1091
Przejazd kolejowy strzeżony                            554
Przejazd tramwajowy, torowisko tramwajowe              493
Parking, plac                                          479
Przystanek komunikacji publicznej                      376
Pas dzielący jezdnie                                   331
Przejazd dla rowerzystów                               289
Droga, pas ruchu, śluza dla rowerów                    2

In [24]:
# Unifikacja wartosci
df['CharMiejsca'] = df['CharMiejsca'].apply(lambda x: 'Parking' if 'Parking' in str(x) else x)
df['CharMiejsca'] = df['CharMiejsca'].apply(lambda x: 'Przejazd rowerowy' if 'Przejazd dla rowerzystów' in str(x) else x)
df['CharMiejsca'] = df['CharMiejsca'].apply(lambda x: 'Sluza dla rowerow' if 'Droga, pas ruchu, śluza dla rowerów' in str(x) else x)
df['CharMiejsca'].value_counts()

Jezdnia                                      191185
Parking                                       19558
Pobocze                                        4676
Przejście dla pieszych                         3624
Chodnik, droga dla pieszych                    2612
Most, wiadukt, łącznica, tunel                 1796
Skarpa, rów                                    1356
Wjazd, wyjazd z posesji, pola                  1322
Roboty drogowe, oznakowanie tymczasowe         1235
Przejazd kolejowy strzeżony                     554
Przejazd tramwajowy, torowisko tramwajowe       493
Przystanek komunikacji publicznej               376
Pas dzielący jezdnie                            331
Przejazd rowerowy                               302
Sluza dla rowerow                               272
Przejazd kolejowy niestrzeżony                  100
Przejazd tramwajowy, torowisko                   66
Przewiązka na drodze dwujezdniowej               50
Droga dla rowerzystów                            19
Name: CharMi

In [25]:
# Oświetlenie miejsca zdarzenia
df['Oswietlenie'].value_counts()
# Dane wystepuja bez powtorzen. Dodam kolumne dzien
df['CzyDzien'] = df['Oswietlenie'].apply(lambda x: True if 'Światło dzienne' in str(x) else False)
df['CzyDzien'].value_counts()

True     171386
False     58543
Name: CzyDzien, dtype: int64

In [26]:
# Warunki atmosferyczne
df['Pogoda'].value_counts()

Dobre warunki atmosferyczne                                   145165
Pochmurno                                                      43516
Opady deszczu                                                  23747
Opady śniegu, gradu                                             6338
Pochmurno; Opady deszczu                                        5104
Oślepiające słońce                                              2848
Mgła, dym                                                       1033
Pochmurno; Opady śniegu, gradu                                   741
Silny wiatr                                                      534
Silny wiatr; Opady deszczu                                       198
Opady deszczu; Opady śniegu, gradu                               158
Silny wiatr; Pochmurno                                           158
Pochmurno; Mgła, dym                                             121
Pochmurno; Opady deszczu; Opady śniegu, gradu                     73
Silny wiatr; Pochmurno; Opady desz

In [27]:
# Podzial na grupy:
# Pochmurno: True/False
df['Zachmurzenie'] = df['Pogoda'].apply(lambda x: True if 'Pochmurno' in str(x) else False)
# Deszcz: True/False
df['Deszcz'] = df['Pogoda'].apply(lambda x: True if 'Opady deszczu' in str(x) else False)
# Snieg/Grad: True/False
df['SniegGrad'] = df['Pogoda'].apply(lambda x: True if 'Opady śniegu, gradu' in str(x) else False)
# Oslepiajace slonce: True/False
df['OslepiajaceSlonce'] = df['Pogoda'].apply(lambda x: True if 'Oślepiające słońce' in str(x) else False)
# Silny wiatr: True/false
df['SilnyWiatr'] = df['Pogoda'].apply(lambda x: True if 'Silny wiatr' in str(x) else False)
# Mgla/dym: True/false
df['MglaDym'] = df['Pogoda'].apply(lambda x: True if 'Mgła, dym' in str(x) else False)

In [28]:
# Rodzaj zdarzenia-wyglada ok
df['RodzajZdarzenia'].value_counts()

Zderzenie pojazdów boczne              77669
Zderzenie pojazdów tylne               59469
Najechanie na pojazd unieruchomiony    29030
Inne                                   16655
Najechanie na zwierzę                  10453
Najechanie na pieszego                  6749
Najechanie na barierę ochronną          6107
Najechanie na słup, znak                5998
Zderzenie pojazdów czołowe              5714
Najechanie na dziurę, wybój, garb       5256
Wywrócenie się pojazdu                  3995
Najechanie na drzewo                    1974
Najechanie na zapore kolejową            580
Zdarzenie z pasażerem                    280
Name: RodzajZdarzenia, dtype: int64

In [29]:
# Wina - ok
df['Wina'].value_counts()

z winy kierującego             198463
z innych przyczyn               25503
współwina uczestników ruchu      3611
z winy pieszego                  2352
Name: Wina, dtype: int64

In [30]:
# Rodzaj drogi
df['RodzajDrogi'].value_counts()

Jednojezdniowa dwukierunkowa    160724
Dwie jezdnie jednokierunkowe     37692
Jednokierunkowa                  20812
Autostrada                        6751
Ekspresowa                        3949
Name: RodzajDrogi, dtype: int64

In [31]:
# Nawierzchnia drogi
df['NawierzchniaDrogi'].value_counts()

Twarda      229380
Gruntowa       548
Name: NawierzchniaDrogi, dtype: int64

In [32]:
# Stan nawierzchni
df['StanNawierzchni'].value_counts()
# Mokra
df['NawierzchniaMokra'] = df['StanNawierzchni'].apply(lambda x: True if 'Mokra' in str(x) else False)
# Sucha
df['NawierzchniaSucha'] = df['StanNawierzchni'].apply(lambda x: True if 'Sucha' in str(x) else False)
# Oblodzona
df['NawierzchniaOblodzona'] = df['StanNawierzchni'].apply(lambda x: True if ('Oblodzenie, zaśnieżenie' in str(x) or 'Oblodzona, zaśnieżona' in str(x)) else False)
# Kaluze, rozlewiska
df['NawierzchniaKaluze'] = df['StanNawierzchni'].apply(lambda x: True if 'Kałuże' in str(x) else False)
# Dziury, wyboje
df['NawierzchniaDziury'] = df['StanNawierzchni'].apply(lambda x: True if 'Dziury' in str(x) else False)
# Zanieczyszczona
df['NawierzchniaZanieczyszczona'] = df['StanNawierzchni'].apply(lambda x: True if 'Zanieczyszczona' in str(x) else False)
# Koleiny, garby
df['NawierzchniaKoleiny'] = df['StanNawierzchni'].apply(lambda x: True if 'Koleiny' in str(x) else False)

In [33]:
df['NawierzchniaKoleiny'].value_counts()

False    229667
True        262
Name: NawierzchniaKoleiny, dtype: int64

In [34]:
df['SygnalizacjaSwietlna'].value_counts()

Brak                207626
Jest, działa         21062
Jest, nie działa      1241
Name: SygnalizacjaSwietlna, dtype: int64

In [35]:
df['ZnakiPoziome'] = df['ZnakiPoziome'].apply(lambda x: True if 'Jest' in str(x) else False)
df['ZnakiPoziome'].value_counts()

True     178816
False     51113
Name: ZnakiPoziome, dtype: int64

In [36]:
df['LiczbaPasow'].value_counts()

2.0      101699
4.0       14937
3.0        9186
1.0        8090
6.0        3577
5.0        2537
8.0         671
7.0         433
10.0        166
12.0        131
9.0         117
50.0         85
11.0         25
20.0         20
18.0         16
13.0         16
14.0         13
16.0         11
70.0          9
0.0           8
60.0          5
30.0          4
40.0          4
90.0          3
66.0          2
22.0          2
17.0          2
250.0         1
67.0          1
24.0          1
15.0          1
25.0          1
32.0          1
Name: LiczbaPasow, dtype: int64

In [37]:
len(df['ZachowanieKierowcy'].unique())

162

In [38]:
df['ZachowanieKierowcy'].value_counts()

Niezachowanie bezp. odleg. między pojazdami                                                                42698
Nieustąpienie pierwszeństwa przejazdu                                                                      34350
Niedostosowanie prędkości do warunków ruchu                                                                23158
Nieprawidłowe cofanie                                                                                      21176
Nieprawidłowe omijanie                                                                                     13811
                                                                                                           ...  
Jazda bez wymaganego oświetlenia; Nieprawidłowe: wyprzedzanie                                                  1
Jazda bez wymaganych świateł; Niedostosowanie prędkości do warunków ruchu                                      1
Gwałtowne hamowanie; Nieustąpienie pierwszeństwa przejazdu                                      

In [39]:
def findRepeatingSubstring(string):
    if string != 'nan':
        for x in range(1, len(string)):
            substring = string[:x]

            if substring * (len(string)//len(substring))+(substring[:len(string)%len(substring)]) == string:
                return re.sub(r"\W+", ' ', substring).rstrip()

In [40]:
df['ZachowanieKierowcy'] = df['ZachowanieKierowcy'].apply(lambda x: findRepeatingSubstring(str(x)).lower().replace(":", "") if findRepeatingSubstring(str(x)) is not None else str(x).lower().replace(":",""))


df['ZachowanieKierowcy'] = df['ZachowanieKierowcy'].apply(lambda x: 'Inne przyczyny' if str(x)=='Inne przyczyny (od 11.2015)' else x)
df['ZachowanieKierowcy'] = df['ZachowanieKierowcy'].apply(lambda x: 'Niestosowanie się do sygnalizacji świetlnej' if str(x)=='Niestosowanie się do sygnalizacji świetlnej (od 11.2015)' else x)
df['ZachowanieKierowcy'] = df['ZachowanieKierowcy'].apply(lambda x: 'Nieustąpienie pierwszeństwa pieszemu na przejściu dla pieszych' if str(x)=='Nieustąpienie pierwszeństwa pieszemu na przejściu dla pieszych (od 11.2015)' else x)
df['ZachowanieKierowcy'] = df['ZachowanieKierowcy'].apply(lambda x: 'Nieustąpienie pierwszeństwa pieszemu w innych okolicznościach' if str(x)=='Nieustąpienie pierwszeństwa pieszemu w innych okolicznościach (od 11.2015)' else x)
df['ZachowanieKierowcy'] = df['ZachowanieKierowcy'].apply(lambda x: 'Nieustąpienie pierwszeństwa pieszemu przy skręcaniu w drogę poprzeczną' if str(x)=='Nieustąpienie pierwszeństwa pieszemu przy skręcaniu w drogę poprzeczną  (od 11.2015)' else x)
df['ZachowanieKierowcy'] = df['ZachowanieKierowcy'].apply(lambda x: 'Omijanie pojazdu przed przejściem dla pieszych' if str(x)=='Omijanie pojazdu przed przejściem dla pieszych (od 11.2015)' else x)
df['ZachowanieKierowcy'] = df['ZachowanieKierowcy'].apply(lambda x: 'Wyprzedzanie pojazdu przed przejściem dla pieszych' if str(x)=='Wyprzedzanie pojazdu przed przejściem dla pieszych  (od 11.2015)' else x)

df['ZachowanieKierowcy'] = df['ZachowanieKierowcy'].apply(lambda x: str(x).replace(" (od 11.2015)", ""))
df['ZachowanieKierowcy'] = df['ZachowanieKierowcy'].apply(lambda x: str(x).replace(" od 11 2015", ""))

df['ZachowanieKierowcy'] = df['ZachowanieKierowcy'].apply(lambda x: 'wyprzedzanie pojazdu przed przejściem dla pieszych' if str(x)=='wyprzedzanie pojazdu przed przejściem dla pieszych ' else x)
df['ZachowanieKierowcy'] = df['ZachowanieKierowcy'].apply(lambda x: 'nieustąpienie pierwszeństwa pieszemu przy skręcaniu w drogę poprzeczną' if str(x)=='nieustąpienie pierwszeństwa pieszemu przy skręcaniu w drogę poprzeczną ' else x)

df['ZachowanieKierowcy'].value_counts().head(20)

niezachowanie bezp. odleg. między pojazdami                       42698
nieustąpienie pierwszeństwa przejazdu                             34414
nan                                                               27901
nieprawidłowe cofanie                                             23745
niedostosowanie prędkości do warunków ruchu                       23195
nieprawidłowe omijanie                                            14969
nieprawidłowe zmienianie pasa ruchu                               13870
nieprawidłowe skręcanie                                           10290
inne przyczyny                                                     9909
nieprawidłowe wymijanie                                            5825
niezachowanie bezp. odl. między pojazdami                          4540
nieprawidłowe wyprzedzanie                                         3782
nieudzielenie pierwszeństwa przejazdu                              3692
nieustąpienie pierwszeństwa pieszemu na przejściu dla pieszych  

In [41]:
len(df['ZachowanieKierowcy'].unique())

113

In [42]:
# Zachowanie pieszego
df['ZachowaniePieszego'] = df['ZachowaniePieszego'].apply(lambda x: 'Inne przyczyny' if 'Inne przyczyny' in str(x) else x)
df['ZachowaniePieszego'] = df['ZachowaniePieszego'].apply(lambda x: 'Nieostrożne wejście na jezdnię: przed jadącym pojazdem' if 'Nieostrożne wejście na jezdnię: przed jadącym pojazdem' in str(x) else x)
df['ZachowaniePieszego'] = df['ZachowaniePieszego'].apply(lambda x: 'Leżenie, siedzenie, klęczenie, stanie na jezdni' if ('Leżenie, siedzenie, klęczenie, stanie na jezdni' in str(x) or 'Stanie na jezdni, leżenie' in str(x)) else x)
df['ZachowaniePieszego'] = df['ZachowaniePieszego'].apply(lambda x: 'Nieostrożne wejście na jezdnię: zza pojazdu, przeszkody' if 'Nieostrożne wejście na jezdnię: zza pojazdu, przeszkody' in str(x) else x)
df['ZachowaniePieszego'] = df['ZachowaniePieszego'].apply(lambda x: 'Przekraczanie jezdni w miejscu niedozwolonym' if 'Przekraczanie jezdni w miejscu niedozwolonym' in str(x) else x)
df['ZachowaniePieszego'] = df['ZachowaniePieszego'].apply(lambda x: 'Chodzenie nieprawidłową stroną drogi' if 'Chodzenie nieprawidłową stroną drogi' in str(x) else x)
df['ZachowaniePieszego'] = df['ZachowaniePieszego'].apply(lambda x: 'Wejście na jezdnię przy czerwonym świetle' if 'Wejście na jezdnię przy czerwonym świetle' in str(x) else x)
df['ZachowaniePieszego'].value_counts()  

Nieostrożne wejście na jezdnię: przed jadącym pojazdem     1270
Nieostrożne wejście na jezdnię: zza pojazdu, przeszkody     362
Inne przyczyny                                              293
Przekraczanie jezdni w miejscu niedozwolonym                287
Wejście na jezdnię przy czerwonym świetle                   156
Chodzenie nieprawidłową stroną drogi                         60
Leżenie, siedzenie, klęczenie, stanie na jezdni              45
Zatrzymanie, cofnięcie się                                   12
Name: ZachowaniePieszego, dtype: int64

In [43]:
# Inne przyczyny
df['InnePrzyczyny'] = df['InnePrzyczyny'].apply(lambda x: 'Niesprawność techniczna pojazdu' if 'Niesprawność techniczna pojazdu' in str(x) else x)
df['InnePrzyczyny'] = df['InnePrzyczyny'].apply(lambda x: 'Utrata przytomności, śmierć kierującego' if 'Utrata przytomności, śmierć kierującego' in str(x) else x)
df['InnePrzyczyny'].value_counts()

Obiekty, zwierzęta na drodze                11603
Nieustalone                                  5876
Stan jezdni                                  4828
Inne                                         2010
Niewłaściwy stan jezdni                       512
Niesprawność techniczna pojazdu               341
Zabezpieczenie robót na drodze                316
Utrata przytomności, śmierć kierującego       266
Z winy pasażera                               218
Pożar pojazdu                                 117
Oślepienie przez inny pojazd lub słońce        71
Organizacja ruchu                              19
Nieprawidłowo zabezp. roboty drogowe           18
Nieprawidłowo działająca zapora kolej.          5
Nieprawidłowo działająca zapora, rogatka        2
Działanie sygnalizacji świetlnej                1
Name: InnePrzyczyny, dtype: int64

In [44]:
df['Ranni'].value_counts()

0     217211
1      10459
2       1581
3        424
4        162
5         50
6         22
7          7
9          5
8          4
26         1
12         1
11         1
10         1
Name: Ranni, dtype: int64

In [45]:
df['Zabici'].value_counts()

0    229059
1       822
2        43
3         4
4         1
Name: Zabici, dtype: int64

In [46]:
df['Kolizja'].value_counts()

1    216531
0     13398
Name: Kolizja, dtype: int64

In [47]:
df['Wypadek'].value_counts()

0    216531
1     13398
Name: Wypadek, dtype: int64

###Spotkanie 3: Zdarzenia w terenie (nie/)zabudowanym: gdzie najwiecej zdarzen, wypadkow, kolizji
Gestosc zaludnienia, powody wypadkow a dzien i godzina [odleglosc od centrum]
